In [2]:
import numpy as np
import scanpy as sc
import pandas as pd
from tool import train
from tool.data import preprocessing,mapping_adj2pyg
import warnings
warnings.filterwarnings("ignore")
from tool.data import mapping_adj2matrix,adata_to_cluster_expression
from tool.calculate_adj import cal_adj,another_cal_adj

DataDir="./data/realdata/mousebrain/"
sc_file_path = DataDir+'scRNA.h5ad'
spatial_file_path = DataDir+'Spatial.h5ad'
RNA_data_adata = sc.read_h5ad(sc_file_path)
Spatial_data_adata = sc.read_h5ad(spatial_file_path)
#RNA_data_adata.obs['celltype']=RNA_data_adata.obs['celltype_final']
celltype_counts = RNA_data_adata.obs['celltype'].value_counts()
print(celltype_counts)

Neurons       5155
Oligos        1009
Astrocytes     822
Vascular       710
Immune         482
Excluded       198
Blood           46
Ependymal       27
Name: celltype, dtype: int64


In [2]:
RNA_ret_adata=RNA_data_adata.copy()
sc.pp.normalize_total(RNA_ret_adata)
RNA_ret_adata=adata_to_cluster_expression(RNA_ret_adata)

In [3]:
print(RNA_data_adata)
print(Spatial_data_adata)
print(RNA_ret_adata)

AnnData object with n_obs × n_vars = 8449 × 27998
    obs: 'celltype'
AnnData object with n_obs × n_vars = 1213 × 19199
AnnData object with n_obs × n_vars = 8 × 27998
    obs: 'celltype', 'cluster_density'


In [4]:
adj_df=another_cal_adj(RNA_ret_adata,Spatial_data_adata,DataDir)
concat_st_sc=preprocessing(RNA_data_adata,Spatial_data_adata)
print(concat_st_sc)

Data exists, load it
load adj dataframe
Drop celltype [] contain less 2 sample
View of AnnData object with n_obs × n_vars = 1220 × 1331
    obs: 'celltype', 'cluster_density', 'n_genes', 'rna_count_based_density', 'batch'
    var: 'n_cells-0', 'n_cells-1'


In [5]:
adj_matrix=mapping_adj2pyg(adj_df,concat_st_sc)
ad_map1=train.start(concat_st_sc,adj_matrix,GCN_method="pygGCN")

Epoch 000: Loss 0.9636978507041931
Epoch 010: Loss 0.842496395111084
Epoch 020: Loss 0.8074029684066772
Epoch 030: Loss 0.7761646509170532
Epoch 040: Loss 0.752677857875824
Epoch 050: Loss 0.7369974255561829
Epoch 060: Loss 0.724661648273468
Epoch 070: Loss 0.7177507877349854
Epoch 080: Loss 0.7105493545532227
Epoch 090: Loss 0.7016606330871582
Epoch 100: Loss 0.6910685300827026
Epoch 110: Loss 0.6791782379150391
Epoch 120: Loss 0.6746663451194763
Epoch 130: Loss 0.6719542741775513
Epoch 140: Loss 0.6705181002616882
Epoch 150: Loss 0.6693385243415833
Epoch 160: Loss 0.6681720018386841
Epoch 170: Loss 0.6672033667564392
Epoch 180: Loss 0.6674150228500366
Epoch 190: Loss 0.6646138429641724
Epoch 200: Loss 0.6630411744117737
Epoch 210: Loss 0.6623125672340393
Epoch 220: Loss 0.6617318391799927
Epoch 230: Loss 0.6611130237579346
Epoch 240: Loss 0.6605608463287354
Epoch 250: Loss 0.6605273485183716
Epoch 260: Loss 0.6609710454940796
Epoch 270: Loss 0.6603024005889893
Epoch 280: Loss 0.65914

In [6]:
ad_map1.obs=ad_map1.obs.dropna(axis=1, how='all')
ad_map1.var=ad_map1.var.dropna(axis=1, how='all')
print(ad_map1)
save_path=DataDir+'/pygGCN_result2.h5ad'
print(save_path)
ad_map1.write(save_path)

AnnData object with n_obs × n_vars = 1212 × 8
    obs: 'n_genes', 'rna_count_based_density', 'batch'
    var: 'celltype', 'cluster_density', 'batch'
./data/realdata/mousebrain//pygGCN_result2.h5ad


In [7]:
adj_matrix=mapping_adj2matrix(adj_df,concat_st_sc)
ad_map=train.start(concat_st_sc,adj_matrix,GCN_method="coGCN")

Epoch 000: Loss 1.0166479349136353
Epoch 010: Loss 1.0
Epoch 020: Loss 1.0
Epoch 030: Loss 1.0
Epoch 040: Loss 1.0
Epoch 050: Loss 1.0


KeyboardInterrupt: 

In [ ]:
ad_map.obs=ad_map.obs.dropna(axis=1, how='all')
ad_map.var=ad_map.var.dropna(axis=1, how='all')
print(ad_map)
save_path=DataDir+'/coGCN_result2.h5ad'
print(save_path)
ad_map.write(save_path)

AnnData object with n_obs × n_vars = 1212 × 8
    obs: 'n_genes', 'rna_count_based_density', 'batch'
    var: 'celltype', 'cluster_density', 'batch'
./data/realdata/mousebrain//coGCN_result1.h5ad
